In [ ]:
import yfinance as yf
import talib
import numpy as np
import requests
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def get_news(token_symbol, api_key):
    url = f'https://newsapi.org/v2/everything?q={token_symbol}&apiKey={api_key}' 
    response = requests.get(url)
    data = response.json()
    return [article['title'] for article in data['articles']]

def get_market_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

def calculate_technical_indicators(data):
    close = data['Close']
    sma = talib.SMA(close, timeperiod=50)
    rsi = talib.RSI(close, timeperiod=14)
    macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    return np.column_stack((sma, rsi, macd))

def prepare_data(symbol, start_date, end_date, token_symbol, api_key):
    market_data = get_market_data(symbol, start_date, end_date)
    
    news = get_news(token_symbol, api_key)
    
    indicators = calculate_technical_indicators(market_data)
    
    market_data['Future Close'] = market_data['Close'].shift(-1)
    market_data['Direction'] = (market_data['Future Close'] > market_data['Close']).astype(int)
    
    features = np.column_stack((indicators, market_data['Direction'].values[:-1]))
    
    return features, market_data['Direction'].values[1:]

def train_model(symbol, start_date, end_date, token_symbol, api_key):
    features, labels = prepare_data(symbol, start_date, end_date, token_symbol, api_key)
    
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    model = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    
    return model

def predict_trend(symbol, token_symbol, model, api_key):
    market_data = get_market_data(symbol, start_date='2023-01-01', end_date='2023-12-31')
    indicators = calculate_technical_indicators(market_data)
    
 
    current_features = np.column_stack((indicators, [0])) 
    prediction = model.predict(current_features[-1].reshape(1, -1))
    

    if prediction == 1:
        print("1: Лонг (Рост рынка)")
    else:
        print("0: Шорт (Падение рынка)")

token_symbol = 'BTC'  
api_key = 'your_newsapi_key'  
symbol = 'BTC-USD'  # Пара для скачивания данных (например, BTC/USDT)


model = train_model(symbol, start_date='2023-01-01', end_date='2023-12-31', token_symbol=token_symbol, api_key=api_key)


predict_trend(symbol, token_symbol, model, api_key)
